In [1]:
import pandas as pd
import numpy as np
from pymystem3 import Mystem
from datetime import datetime
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [2]:
dfcolumns = pd.read_csv(r'ВКР/Kalininsky_post_1.csv',
                        nrows = 1, delimiter=';', encoding = 'UTF-8')
df = pd.read_csv(r'ВКР/Kalininsky_post_1.csv',
                  header = None,
                  delimiter=';',
                  skiprows = 1,
                  encoding='UTF-8',
                  usecols = list(range(len(dfcolumns.columns))),
                  names = dfcolumns.columns)

In [3]:
length = len(df)
print(length)

43959


In [4]:
df.head()[:1]

,ID,ТИП,ПОДТИП,ССЫЛКА НА ПОСТ,НАЗВАНИЕ ВЛАДЕЛЬЦА,ПОДПИСЧИКОВ,ССЫЛКА НА ВЛАДЕЛЬЦА СТЕНЫ,ССЫЛКА НА АВТОРА ПОСТА,АВТОРСТВО,ССЫЛКА НА ПОСТ С УЧЁТОМ АВТОРА,ДАТА ПУБЛИКАЦИИ,ТЕКСТ,ЛАЙКОВ,РЕПОСТОВ,КОММЕНТАРИЕВ,ПРОСМОТРОВ,ВНЕШНИЙ_ИСТОЧНИК,ИСТОЧНИК,НАЗВАНИЕ ИСТОЧНИКА
0,1,official,NaN,https://vk.com/wall-23904_314207,Калининский район Санкт-Петербурга,65311,https://vk.com/club23904,https://vk.com/club23904,1,https://vk.com/club23904?w=wall-23904_314207,27.04.2023,Сегодня в 14:00 состоится прямой эфир из Пресс...,51.0,14.0,118.0,5853.0,0,NaN,NaN


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43959 entries, 0 to 43958
Data columns (total 19 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   ID                              43959 non-null  int64  
 1   ТИП                             43959 non-null  object 
 2   ПОДТИП                          0 non-null      float64
 3   ССЫЛКА НА ПОСТ                  43959 non-null  object 
 4   НАЗВАНИЕ ВЛАДЕЛЬЦА              43959 non-null  object 
 5   ПОДПИСЧИКОВ                     43959 non-null  int64  
 6   ССЫЛКА НА ВЛАДЕЛЬЦА СТЕНЫ       43959 non-null  object 
 7   ССЫЛКА НА АВТОРА ПОСТА          43959 non-null  object 
 8   АВТОРСТВО                       43959 non-null  int64  
 9   ССЫЛКА НА ПОСТ С УЧЁТОМ АВТОРА  43959 non-null  object 
 10  ДАТА ПУБЛИКАЦИИ                 43959 non-null  object 
 11  ТЕКСТ                           38932 non-null  object 
 12  ЛАЙКОВ                          

In [6]:
cols = [col for col in df.columns]
print(cols)

['ID', 'ТИП', 'ПОДТИП', 'ССЫЛКА НА ПОСТ', 'НАЗВАНИЕ ВЛАДЕЛЬЦА', 'ПОДПИСЧИКОВ', 'ССЫЛКА НА ВЛАДЕЛЬЦА СТЕНЫ', 'ССЫЛКА НА АВТОРА ПОСТА', 'АВТОРСТВО', 'ССЫЛКА НА ПОСТ С УЧЁТОМ АВТОРА', 'ДАТА ПУБЛИКАЦИИ', 'ТЕКСТ', 'ЛАЙКОВ', 'РЕПОСТОВ', 'КОММЕНТАРИЕВ', 'ПРОСМОТРОВ', 'ВНЕШНИЙ_ИСТОЧНИК', 'ИСТОЧНИК', 'НАЗВАНИЕ ИСТОЧНИКА']


In [7]:
import pandas as pd
import openpyxl

# читаем файл Excel
wb = openpyxl.load_workbook('ВКР/Base.xlsx')
ws = wb.active


# получаем значения столбцов H, C и D - можно через ord, например: ord('C')-64
links = [cell.value for row in ws.iter_rows(min_row=2, max_row=56, min_col=8, max_col=8) for cell in row] # H - 8, строки с 2 по 56
types = [cell.value for row in ws.iter_rows(min_row=2, max_row=56, min_col=3, max_col=3) for cell in row]
subtypes = [cell.value for row in ws.iter_rows(min_row=2, max_row=56, min_col=4, max_col=4) for cell in row]

# присваиваем значения столбца 'type' в зависимости от значения столбца 'link'
for i, row in df.iterrows():
    if row['ССЫЛКА НА ВЛАДЕЛЬЦА СТЕНЫ'] in links:
        index = links.index(row['ССЫЛКА НА ВЛАДЕЛЬЦА СТЕНЫ'])
        df.at[i, 'ТИП'] = types[index]
        df.at[i, 'ПОДТИП'] = subtypes[index]



In [1]:
df

NameError: name 'df' is not defined

In [9]:
df['ТЕКСТ2'] = df['ТЕКСТ']

In [10]:
import re

print("Cleaning text...")
with open("C:/myR/stopwords.txt", "r", encoding="UTF-8") as file:
    stop_words = set(map(str.strip, file))


df['ТЕКСТ2'] = df['ТЕКСТ']
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.lower().str.replace("<[^>]*>", " ", regex=True) #Удаляем знаки
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("\r|\n", "", regex=True) #Удаляем переходы на новую строку
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("\[[^[]*\|", "", regex=True)
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("[^\w\s]", " ", regex=True)
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("ё", "е", regex=True) #Заменяем "ё" на "е"
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("\s\s+", " ", regex=True)
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("\d", "", regex=True) #Удаляем цифры
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("  ", " ", regex=True) #Удаляем двойные пробелы
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.strip() #Удаляем пробелы в обоих концах строк
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("[a-z]", "", regex=True)
df['ТЕКСТ2'] = df['ТЕКСТ2'].str.replace("[A-Z]", "", regex=True)


# Удаляем стоп-слова
df['ТЕКСТ2'] = df['ТЕКСТ2'].apply(lambda x: ' '.join(filter(lambda word: word not in stop_words, x.split())) if isinstance(x, str) else x)

print("Successful!")
print("---------------------------------")



Cleaning text...
Successful!
---------------------------------


In [11]:
df['ТЕКСТ2']

0        состоится прямой эфир пресс студии калининског...
1          площадь ленина год фото архива сергея потемкина
2        продолжаем квн первая игра лиги квн малая нева...
3        уличные выступления неотъемлемая часть петербу...
4        центральная районная библиотека белинского цен...
                               ...                        
43954    устремленные солнцу побеги синих высот выраста...
43955    жители домов рядом парком сахарова жалуются пы...
43956    друзья прямо умельцы справившиеся забором квс ...
43957    друзья разбавим трудовое утро наверняка тронет...
43958    друзья дворах рядом стройкой разворачивается т...
Name: ТЕКСТ2, Length: 43959, dtype: object

In [12]:
df['ТЕКСТ3'] = df['ТЕКСТ2']

In [16]:
from concurrent.futures import ThreadPoolExecutor
from pymystem3 import Mystem

### Здесь мы создаем функцию lemmatize_text, которая принимает на вход текст и возвращает лемматизированный текст 

def lemmatize_text(text):                         
    m = Mystem()
    lemmas = m.lemmatize(text)                       #лемматизируем текст
    list_len = len(lemmas)                           #узнаем количество слов в list'е lemmas (удвоенное)
    if list_len > 0:                                 #проверяем есть ли слова в посте
        if list_len == 2:                            #проверяем больше ли одного слова в посте
            mystr = lemmas[0]                        #приравниваем строку для записи в csv файл с единственным словом в посте
        elif list_len > 2:                           # == если в посте больше двух слов
            list_len -= 2                            # кол-во элементов в списке на 1 больше индекса последнего элемента списка;
            mystr = ""                               # последний элемент данного списка == '\n'
            i = 0
            while i <= list_len:                       #цикл считывания лемм
                mystr += lemmas[i] + " "               #прибавляем слова к строке для записи с пробелом на конце
                i += 2                               #так как в списке все слова имеют четный индекс, то прибавляем 2
        return mystr.strip()
    else:
        return ""                               


    
def process_chunk(chunk):                             # каждый chunk состояит из 1/n куска df
    
    for i, row in chunk.iterrows():
        text = row['ТЕКСТ2']
        if isinstance(text, str):
            lemmatized_text = lemmatize_text(text)
            df.at[i, 'ТЕКСТ3'] = lemmatized_text
            
      

    
def process_data_in_parallel(data, num_threads):               # распараллеливание процесса
    chunk_size = len(data) // num_threads
    chunks = [data.iloc[i:i+chunk_size] for i in range(0, len(data), chunk_size)]
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        executor.map(process_chunk, chunks)


print("Lemmatizing text...") 
process_data_in_parallel(df, 4)        # 4 - n - количество параллельных процессов
print(' ')
print("Successful!")
print("---------------------------------")

Lemmatizing text...
 
Successful!
---------------------------------


In [17]:
df['ТЕКСТ3']

0        состояться прямой эфир пресс студия калинински...
1             площадь ленин год фото архив сергей потемкин
2        продолжать квн первый игра лига квн малый нева...
3        уличный выступление неотъемлемый часть петербу...
4        центральный районный библиотека белинский цент...
                               ...                        
43954    устремленные солнцу побеги синих высот выраста...
43955    жители домов рядом парком сахарова жалуются пы...
43956    друг прямо умелец справляться забор квс пытать...
43957    друг разбавлять трудовой утро наверняка тронут...
43958    друг двор рядом стройка разворачиваться трагик...
Name: ТЕКСТ3, Length: 43959, dtype: object

In [18]:
import nltk         #  pip install nltk
import pymorphy2    #  pip install pymorphy2 pymorphy2-dicts
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\msvic\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [26]:
text

NameError: name 'text' is not defined

In [29]:
morph = pymorphy2.MorphAnalyzer()        #создать экземпляр класса MorphAnalyzer.


#Создать функцию count_imperative_verbs, которая будет принимать текст и экземпляр класса MorphAnalyzer, 
#а затем использовать его для определения частей речи и проверки, содержится ли глагол в повелительном наклонении.

def count_imperative_verbs(text):
    if isinstance(text, str):
        words = text.split()
        count = 0
        for word in words:
            parsed_word = morph.parse(word)[0]
            if 'VERB' in parsed_word.tag and 'impr' in parsed_word.tag:
                count += 1
        return count
    else:
        return 0

def count_past_verbs(text):
    if isinstance(text, str):
        words = text.split()
        count = 0
        for word in words:
            parsed_word = morph.parse(word)[0]
            if 'VERB' in parsed_word.tag and 'past' in parsed_word.tag:
                count += 1
        return count
    else:
        return 0


def count_punctuation(text, punctuation):
    if isinstance(text, str):
        return text.count(punctuation)
    else:
        return 0
    
def count_len(text):
    if isinstance(text, str):
        return len(text)
    else:
        return 0
        

df['!'] = df['ТЕКСТ'].apply(lambda x: count_punctuation(x, '!'))
df['?'] = df['ТЕКСТ'].apply(lambda x: count_punctuation(x, '?'))
df['...'] = df['ТЕКСТ'].apply(lambda x: count_punctuation(x, '...'))
df['ДЛИНА'] = df['ТЕКСТ3'].apply(lambda x: count_len(x))
df['ИМПЕРАТИВ'] = df['ТЕКСТ2'].apply(lambda x: count_imperative_verbs(x))
df['ПРОШЕДШЕЕ'] = df['ТЕКСТ2'].apply(lambda x: count_past_verbs(x))

In [2]:
df

NameError: name 'df' is not defined

In [ ]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)
print("Calculating tonalities...")

In [45]:
dat_text = df['ТЕКСТ2'].tolist()
results = []
for text in dat_text:
    if isinstance(text, str):
        result = model.predict(text, k=3)
        results.append(result)
    else:
        results.append(None)

In [42]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)
print("Calculating tonalities...")

dat_text = df['ТЕКСТ2']
results = model.predict(str(dat_text), k=3)

print("Successful!")
print("---------------------------------")

Calculating tonalities...
Successful!
---------------------------------


In [48]:
df[['ПОЗИТИВНОСТЬ', 'НЕЙТРАЛЬНОСТЬ', 'НЕГАТИВНОСТЬ', 'СЕНТИМЕНТ', 'ТОНАЛЬНОСТЬ']] = 0
numofrows = df.shape[0] #Количество строк в файле
dat_text = list(df['ТЕКСТ2'])

In [22]:
numofrows

13

In [ ]:
dat_pos = df['ПОЗИТИВНОСТЬ']
dat_neu = df['НЕЙТРАЛЬНОСТЬ']
dat_neg = df['НЕГАТИВНОСТЬ'] 
print("Saving results of prediction...")
        


In [ ]:
def process_row(i):
    if type(dat_text[i]) is str:
        try:
            df.loc[i, 'ПОЗИТИВНОСТЬ'] = results[i]['positive']
        except KeyError:
            df.loc[i, 'ПОЗИТИВНОСТЬ'] = 0
        try:
            df.loc[i, 'НЕЙТРАЛЬНОСТЬ'] = results[i]['neutral']
        except KeyError:
            df.loc[i, 'НЕЙТРАЛЬНОСТЬ'] = 0
        try:
            df.loc[i, 'НЕГАТИВНОСТЬ'] = results[i]['negative']
        except KeyError:
            df.loc[i, 'НЕГАТИВНОСТЬ'] = 0
    else:
        df.loc[i, 'ПОЗИТИВНОСТЬ'] = 0
        df.loc[i, 'НЕЙТРАЛЬНОСТЬ'] = 0
        df.loc[i, 'НЕГАТИВНОСТЬ'] = 0

    if (dat_pos[i] > dat_neu[i]) and (dat_pos[i] > dat_neg[i]):
        df.loc[i, 'СЕНТИМЕНТ'] = 1
        df.loc[i, 'ТОНАЛЬНОСТЬ'] = dat_pos[i]
    elif (dat_neu[i] > dat_neg[i]):
        df.loc[i, 'СЕНТИМЕНТ'] = 2
        df.loc[i, 'ТОНАЛЬНОСТЬ'] = dat_neu[i]
    else:
        df.loc[i, 'СЕНТИМЕНТ'] = 3
        df.loc[i, 'ТОНАЛЬНОСТЬ'] = dat_neg[i]
        
        
with ThreadPoolExecutor(max_workers=4) as executor:
    executor.map(process_row, range(numofrows))



print("\nSuccessful!")
print("---------------------------------")

In [ ]:
df

In [ ]:
# сохранение измененного файла csv
df.to_csv('POSTS_KALIN.csv', sep=';')

In [87]:
i

1

In [81]:
process_text(subset['ТЕКСТ3'][1])

['площадь', 'ленин', 'год', 'фото', 'архив', 'сергей', 'потемкин']

In [74]:
subset

,ID,ТИП,ПОДТИП,ССЫЛКА НА ПОСТ,НАЗВАНИЕ ВЛАДЕЛЬЦА,ПОДПИСЧИКОВ,ССЫЛКА НА ВЛАДЕЛЬЦА СТЕНЫ,ССЫЛКА НА АВТОРА ПОСТА,АВТОРСТВО,ССЫЛКА НА ПОСТ С УЧЁТОМ АВТОРА,...,!,?,...,ИМПЕРАТИВ,ПРОШЕДШЕЕ,ПОЗИТИВНОСТЬ,НЕЙТРАЛЬНОСТЬ,НЕГАТИВНОСТЬ,СЕНТИМЕНТ,ТОНАЛЬНОСТЬ
0,1,official,dist,https://vk.com/wall-23904_314207,Калининский район Санкт-Петербурга,65311,https://vk.com/club23904,https://vk.com/club23904,1,https://vk.com/club23904?w=wall-23904_314207,...,0,0,0,0,1,0.000000,0.877487,0.129413,3,0
1,2,official,dist,https://vk.com/wall-23904_314705,Калининский район Санкт-Петербурга,65311,https://vk.com/club23904,https://vk.com/club23904,1,https://vk.com/club23904?w=wall-23904_314705,...,0,0,0,0,0,0.000000,0.672342,0.071601,3,0
2,3,official,dist,https://vk.com/wall-23904_314702,Калининский район Санкт-Петербурга,65311,https://vk.com/club23904,https://vk.com/club23904,1,https://vk.com/club23904?w=wall-23904_314702,...,2,0,0,0,1,0.245095,0.422515,0.000000,3,0
3,4,official,dist,https://vk.com/wall-23904_314698,Калининский район Санкт-Петербурга,65311,https://vk.com/club23904,https://vk.com/club23904,1,https://vk.com/club23904?w=wall-23904_314698,...,0,0,0,0,3,0.092698,0.629785,0.228166,3,0
4,5,official,dist,https://vk.com/wall-23904_314695,Калининский район Санкт-Петербурга,65311,https://vk.com/club23904,https://vk.com/club23904,1,https://vk.com/club23904?w=wall-23904_314695,...,3,0,0,0,0,0.069552,0.712242,0.125933,3,0
5,6,official,dist,https://vk.com/wall-23904_314692,Калининский район Санкт-Петербурга,65311,https://vk.com/club23904,https://vk.com/club23904,1,https://vk.com/club23904?w=wall-23904_314692,...,4,0,0,2,0,0.173298,0.554480,0.069552,3,0
6,7,official,dist,https://vk.com/wall-23904_314690,Калининский район Санкт-Петербурга,65311,https://vk.com/club23904,https://vk.com/club23904,1,https://vk.com/club23904?w=wall-23904_314690,...,0,0,0,0,6,0.206904,0.437833,0.196836,3,0
7,8,official,dist,https://vk.com/wall-23904_314688,Калининский район Санкт-Петербурга,65311,https://vk.com/club23904,https://vk.com/club23904,1,https://vk.com/club23904?w=wall-23904_314688,...,1,11,0,0,19,0.148057,0.658428,0.160276,3,0
8,9,official,dist,https://vk.com/wall-23904_314682,Калининский район Санкт-Петербурга,65311,https://vk.com/club23904,https://vk.com/club23904,1,https://vk.com/club23904?w=wall-23904_314682,...,8,2,0,0,10,0.196836,0.300756,0.100889,3,0
9,10,official,dist,https://vk.com/wall-23904_314676,Калининский район Санкт-Петербурга,65311,https://vk.com/club23904,https://vk.com/club23904,1,https://vk.com/club23904?w=wall-23904_314676,...,0,0,0,1,2,0.073706,0.665421,0.201823,3,0
